In [1]:
# Packages needed
import pandas as pd
from dataprep.clean import clean_country

In [2]:
# Paths to datasets
train_data = "datasets_hi4/train-data.csv"
GSCPI_data = "datasets_hi4/extra-dataset/GSCPI_data.csv"
LPI_data = "datasets_hi4/extra-dataset/LPIextend.csv"
country_code = "datasets_hi4/extra-dataset/country_code.csv"

In [3]:
# Initializing datasets
td = pd.read_csv(train_data, sep=";")
GSCPId = pd.read_csv(GSCPI_data, sep=",")
LPId = pd.read_csv(LPI_data, sep=",")

In [4]:
# Convert last column to int
td = td.dropna(subset=["Month 1"])
for column in ["Month 1", "Month 2", "Month 3", "Month 4"]:
    td[column] = td[column].str.replace(" ", "").astype(float)

In [5]:
td.drop_duplicates(inplace=True)

In [6]:
# Merging LPI Mean score to train data
columns = []
for column in LPId.columns:
    if "Score" in column:
        columns.append(column)
LPId["LPI Mean Score"] = LPId[columns].mean(axis=1)
LPId = clean_country(LPId, "Country", output_format="alpha-2", inplace=True)
LPId = LPId[["Country_clean", "LPI Mean Score"]].rename(columns={"Country_clean": "Country"})
td = pd.merge(td, LPId, on = "Country")

/home/alper/anaconda3/lib/python3.8/site-packages/dask/dataframe/core.py:6597: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


  0%|                                                     | 0/8 [00:00<?, ?it/s]

Country Cleaning Report:
	248 values cleaned (99.2%)
	2 values unable to be parsed (0.8%), set to NaN
Result contains 248 (99.2%) values in the correct format and 2 null values (0.8%)


In [7]:
month_lib = {
    "jan": 0,
    "feb": 1,    
    "mar": 2,
    "apr": 3,    
    "may": 4,
    "jun": 5,    
    "jul": 6,
    "aug": 7,    
    "sep": 8,
    "oct": 9,    
    "nov": 10,
    "dec": 11
}

In [8]:
date = td["Date"].str.split(n=1, expand=True)
td["Year"] = date[1]
td["Quarter"] = pd.to_numeric(date[0].str.slice(stop=3).replace(month_lib))//3
td["Quarter"] = td["Quarter"].astype(str)

In [9]:
GSCPId["Year"] = GSCPId["Year-Month"].str.slice(stop=4)
GSCPId["Quarter"] = pd.to_numeric(GSCPId["Year-Month"].str.slice(start=5))//3
GSCPId["Quarter"] = GSCPId["Quarter"].astype(str)

In [10]:
GSCPId = GSCPId.groupby(["Year", "Quarter"], as_index=False)["GSCPI"].mean()

In [11]:
td = pd.merge(td, GSCPId, on=["Year", "Quarter"])

In [13]:
td.to_csv("datasets_hi4/train_data_extra.csv",sep=";")